 INSPECT TOPOLOGICAL PROPERTIES OF TORUS:
 - CREATE THE PROPER POINT CLOUD
 - INIT AN ALPHA-COMPLEX or VIETORIS-RIPS COMPLEX
 - COMPUTE THE PERSISTENCE DIAGRAM
 - INSPECT THE TRIANGULARIZATION


In [ ]:
import numpy as np

def generate_random_points_on_torus(num_points, major_radius, minor_radius):
    # Generate random angles for theta and phi
    theta = np.random.uniform(0, 2*np.pi, num_points)
    phi = np.random.uniform(0, 2*np.pi, num_points)
    
    # Convert spherical coordinates to Cartesian coordinates
    x = (major_radius + minor_radius * np.cos(phi)) * np.cos(theta)
    y = (major_radius + minor_radius * np.cos(phi)) * np.sin(theta)
    z = minor_radius * np.sin(phi)
    
    return np.stack((x, y, z), axis=-1)

# Define parameters of the torus
major_radius = 2.0
minor_radius = 1.0
num_points = 3000

# Generate random points on the torus
points_on_torus = generate_random_points_on_torus(num_points, major_radius, minor_radius)

print("Generated random points on the torus shape:", points_on_torus.shape)


In [ ]:
import plotly.graph_objs as go

# Create trace for the point cloud
trace = go.Scatter3d(
    x=points_on_torus[:, 0],
    y=points_on_torus[:, 1],
    z=points_on_torus[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='rgb(255,0,0)',  # Set color to red
        opacity=0.8
    )
)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show interactive plot
fig.show()

# ALPHA COMPLEXES ON THE POINT CLOUD

In [ ]:
import gudhi as gd
ac = gd.AlphaComplex(points=points_on_torus) # alpha-complex with alpha=0.005 by default.
st = ac.create_simplex_tree() # param to modify the default alpha: max_alpha_square = 0.2**2
print(f"Dimension {st.dimension()}\nNum of simplicies {st.num_simplices()}\nNum of vertices {st.num_vertices()}")

points = np.array([ac.get_point(i) for i in range(st.num_vertices())])
print(f"Example of points: \n{points[:3]}")

# We want to plot triangles of the alpha-complex
triangles = np.array([s[0] for s in st.get_skeleton(2) if len(s[0]) == 3 and s[1] <= 0.005])
print(f"Number of triangles {len(triangles)}")

simplex, filtr_val = next(iter(st.get_simplices()))
print(f"Example of a simplex: {simplex} with filtration value {filtr_val}")

In [ ]:
BarCodes_AC = st.persistence()
print(f"Len of persistence points: {len(BarCodes_AC)}")
print(f"first {BarCodes_AC[0]}\n...\nlast {BarCodes_AC[-1]}\n")
gd.plot_persistence_diagram(BarCodes_AC)

In [ ]:
# Visualization with plotly

import plotly
from plotly.graph_objs import graph_objs as go
import ipywidgets as widgets

plotly.offline.init_notebook_mode()
from plotly.offline import iplot

alpha = widgets.FloatSlider(
    value = 0.05,
    min = 0.0,
    max = 0.1,
    step = 0.0001,
    description = 'Alpha:', 
    readout_format = '.4f'
)

mesh = go.Mesh3d(
    x = points[:, 0], 
    y = points[:, 1], 
    z = points[:, 2], 
    i = triangles[:, 0], 
    j = triangles[:, 1], 
    k = triangles[:, 2]
)

fig = go.FigureWidget(
    data = mesh, 
    layout = go.Layout(
        title = dict(
            text = 'Alpha Complex Representation of the 2-Torus'
        ), 
        scene = dict(
            xaxis = dict(nticks = 4, range = [-4., 4.]), 
            yaxis = dict(nticks = 4, range = [-4., 4.]), 
            zaxis = dict(nticks = 4, range = [-2., 2.])
        )
    )
)

def view_torus(alpha):
    if alpha < 0.0015:
        alpha = 0.0015
    triangles = np.array([s[0] for s in st.get_skeleton(2) if len(s[0]) == 3 and s[1] <= alpha])
    fig.data[0].i = triangles[:, 0]
    fig.data[0].j = triangles[:, 1]
    fig.data[0].k = triangles[:, 2]
    iplot(fig)

widgets.interact(view_torus, alpha = alpha);

# VIETORIS RIPS ON POINTS CLOUD

In [ ]:
import gudhi as gd
max_len = 0.8 # maximal diameter

skeleton = gd.RipsComplex(points = points_on_torus, max_edge_length = max_len)
# topological graph with:
# as many vertices as there are points;
# as edges only pairs of points whose distance is smaller than or equal to 'max_edge_length.
rips = skeleton.create_simplex_tree(max_dimension = 3)
print(f"{rips.dimension()}, {rips.num_vertices()}, {rips.num_simplices()}") 

In [ ]:
BarCodes_Rips = rips.persistence()
print(f"Len of persistence points: {len(BarCodes_Rips)}")
print(f"first {BarCodes_Rips[0]}\n...\nlast {BarCodes_Rips[-1]}\n")
gd.plot_persistence_diagram(BarCodes_Rips)

In [2]:
import numpy as np
import gudhi.representations as gr

# Example persistence diagram
persistence_diagram = [
    (0, 0.5),  # Birth and death of features
    (1, 2.0),
    (1.5, 2.5)
]

# Convert to GUDHI Persistence Diagram Format
diag = np.array([[b, d] for b, d in persistence_diagram])

# 1. Compute Total Persistence
p = 2  # Use p = 2 for squared total persistence
total_persistence = np.sum([(d - b) ** p for b, d in diag])
print("Total Persistence:", total_persistence)

# 2. Compute Persistence Landscape
landscape = gr.Landscape(resolution=1000)  # Note: "gr.Landscape" is used instead of "PersistenceLandscape"
landscape.fit([diag])
landscape_values = landscape.transform([diag])[0]  # Extract the landscape for the first diagram
print("Persistence Landscape (as array):", landscape_values)

# 3. Compute Persistence Image
persistence_image = gr.PersistenceImage(bandwidth=0.1, weight=lambda x: x[1], resolution=[10, 10])
persistence_image.fit([diag])
image = persistence_image.transform([diag])[0]  # Extract the image for the first diagram
print("Persistence Image (as array):", image)


Total Persistence: 2.25
Persistence Landscape (as array): [0.003532   0.007064   0.01059601 ... 0.         0.         0.        ]
Persistence Image (as array): [7.95774715e+00 1.98428183e+00 3.07640150e-02 2.96559847e-05
 2.31071096e-07 1.47894604e-05 5.93117484e-05 1.50187540e-05
 1.50187540e-05 5.93117484e-05 6.81976457e+00 1.70052336e+00
 2.63646652e-02 2.54179439e-05 3.16194323e-06 2.03847381e-04
 8.17510866e-04 2.07007800e-04 2.07007800e-04 8.17510866e-04
 4.29246856e+00 1.07033652e+00 1.65943406e-02 1.60273821e-05
 3.19940419e-05 2.06355684e-03 8.27570182e-03 2.09554992e-03
 2.09554992e-03 8.27570182e-03 1.98428183e+00 4.94785057e-01
 7.67107510e-03 7.62402380e-06 2.37862893e-04 1.53421502e-02
 6.15282593e-02 1.55800126e-02 1.55800126e-02 6.15282593e-02
 6.73686868e-01 1.67985309e-01 2.60441971e-03 3.76264327e-06
 1.29883573e-03 8.37750155e-02 3.35971869e-01 8.50738510e-02
 8.50738510e-02 3.35971869e-01 1.67985309e-01 4.18875077e-02
 6.49418089e-04 5.64721759e-06 5.20883930e-03 3